In [1]:
# import DiCE
import dice_ml
from dice_ml.utils import helpers # helper functions

# Tensorflow libraries
import tensorflow as tf
from tensorflow import keras

In [2]:
import numpy as np
import pandas as pd
import sklearn
from sklearn import datasets, linear_model, preprocessing

In [3]:
names = ['id','thick','size','shape','marg','cell_size','bare',
         'chrom','normal','mit','class']
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/' +
                 'breast-cancer-wisconsin/breast-cancer-wisconsin.data',
                names=names,na_values='?',header=None)

df.drop(columns=['id'],inplace=True)

df = df.dropna()
df.head(6)

# 2 is benign, 4 is malignant

,thick,size,shape,marg,cell_size,bare,chrom,normal,mit,class
0,5,1,1,1,2,1.0,3,1,1,2
1,5,4,4,5,7,10.0,3,2,1,2
2,3,1,1,1,2,2.0,3,1,1,2
3,6,8,8,1,3,4.0,3,7,1,2
4,4,1,1,3,2,1.0,3,1,1,2
5,8,10,10,8,7,10.0,9,7,1,4


In [4]:
df.dtypes

thick          int64
size           int64
shape          int64
marg           int64
cell_size      int64
bare         float64
chrom          int64
normal         int64
mit            int64
class          int64
dtype: object

In [5]:
df['bare'].unique()

array([ 1., 10.,  2.,  4.,  3.,  9.,  7.,  5.,  8.,  6.])

In [6]:
#Turning diagnosis from 4/2 to 1/0
#df['class']=df['class'].map({4:1,2:0})
#df.head(6)

In [7]:
#Making all values ints
#df = df.astype(int)
#df.head(5)

In [8]:
#from numpy.random import seed
#seed(1)
#tf.random.set_seed(2)

In [9]:
d = dice_ml.Data(dataframe=df, continuous_features=['thick','size','shape','marg','cell_size','bare',
         'chrom','normal','mit'],outcome_name='class')

In [10]:
#train, _ = d.split_data(d.normalize_data(d.one_hot_encoded_data))
#xtr = train.loc[:, train.columns != 'class']
#ytr = train.loc[:, train.columns == 'class']

#print(xtr)

In [11]:
x=df[['thick','size','shape','marg','cell_size','bare','chrom','normal','mit']].values
y=df['class'].values

from sklearn.model_selection import train_test_split

xtr, xts, ytr, yts = train_test_split(x,y, test_size=0.1)


In [12]:
#NEURAL NET MODEL

import os
import tensorflow as tf
import tensorflow.keras.backend as K

K.clear_session()

model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation='relu', input_shape=(xtr.shape[1],), name='hidden'),
  tf.keras.layers.Dense(1, activation='sigmoid', name='output') #sigmoid output=binary classification
])


from tensorflow.keras import optimizers

opt = optimizers.Adam(lr=0.01) 
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit( xtr, ytr, batch_size = 3, epochs = 10, shuffle = True,verbose=False)#, validation_data=(xts,yts))


In [13]:
backend = 'TF'+tf.__version__[0] # TF2
m = dice_ml.Model(model=model, backend=backend)

In [14]:
# initiate DiCE
exp = dice_ml.Dice(d, m)

In [15]:
# query instance in the form of a dictionary; keys: feature name, values: feature value


query_instance = {'thick':9, 
                  'size':6, 
                  'shape':9, 
                  'marg':2, 
                  'cell_size':10,
                  'bare': 6.0, 
                  'chrom':2, 
                  'normal': 9,
                  'mit':4}
#data point 98, result is 1/malignant

In [16]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=1,desired_class="opposite")

/Users/ninapiesanen/VIDA/counterfactual_explorer/env/lib/python3.7/site-packages/dice_ml/dice_interfaces/dice_tensorflow2.py:147: RuntimeWarning: divide by zero encountered in double_scalars
  feature_weights[feature] = round(1/normalized_mads[feature], 2)
/Users/ninapiesanen/VIDA/counterfactual_explorer/env/lib/python3.7/site-packages/dice_ml/dice_interfaces/dice_tensorflow2.py:147: RuntimeWarning: divide by zero encountered in true_divide
  feature_weights[feature] = round(1/normalized_mads[feature], 2)


Diverse Counterfactuals found! total time taken: 00 min 41 sec


In [17]:
dice_exp.visualize_as_dataframe()

Query instance (original outcome : 1)


,thick,size,shape,marg,cell_size,bare,chrom,normal,mit,class
0,9.0,6.0,9.0,2.0,10.0,6.0,2.0,9.0,4.0,1.0



Diverse Counterfactual set (new outcome : 0)


,thick,size,shape,marg,cell_size,bare,chrom,normal,mit,class
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
